#Environment setup — install deps, set Gemini key, disable Vertex, verify ADK CLI


In [1]:
# Deps for tools (idempotent)
!pip -q install --upgrade google-adk requests wikipedia langchain-community

import os, sys
print("Python:", sys.version)

# Set your Google AI Studio key
os.environ["GOOGLE_API_KEY"] = ""
os.environ["GEMINI_API_KEY"]  = os.environ["GOOGLE_API_KEY"]  # harmless mirror

# Ensure ADK does NOT route to Vertex (OAuth-only)
for var in ["GOOGLE_GENAI_USE_VERTEXAI","GOOGLE_VERTEX_PROJECT","GOOGLE_VERTEX_LOCATION","GOOGLE_CLOUD_PROJECT"]:
    os.environ.pop(var, None)

print("Key set (masked):", os.environ["GOOGLE_API_KEY"][:6] + "****")

# Sanity: CLI visible?
!adk --help | tail -n +10


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.0 which is incompatible.
langchain 0.3.27 requires langchain-text-splitters<1.0.0,>=0.3.9, but you have langchain-text-splitters 1.0.0 which is incompatible.
Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [G

#Scaffold A2 tools agent package (a2_tools_agent)


In [2]:
%%bash
set -e
rm -rf a2_tools_agent
mkdir -p a2_tools_agent

cat > a2_tools_agent/__init__.py << 'PY'
from .agent import root_agent
PY

echo "[OK] created a2_tools_agent/"
ls -la a2_tools_agent


[OK] created a2_tools_agent/
total 12
drwxr-xr-x 2 root root 4096 Oct 20 03:52 .
drwxr-xr-x 1 root root 4096 Oct 20 03:52 ..
-rw-r--r-- 1 root root   30 Oct 20 03:52 __init__.py


#Write .env so `adk run` inherits the API key


In [3]:
# Ensure CLI picks up the key when it spawns a new process
import pathlib, os
pathlib.Path("a2_tools_agent/.env").write_text(f"GOOGLE_API_KEY={os.environ['GOOGLE_API_KEY']}\n")
print("Wrote a2_tools_agent/.env (key not printed)")


Wrote a2_tools_agent/.env (key not printed)


#Tool: get_fx_rate — currency exchange via free REST APIs


In [4]:
%%bash
set -e
cat > a2_tools_agent/custom_functions.py << 'PY'
import requests

def get_fx_rate(base: str, target: str):
    """
    Fetch the current exchange rate between two currencies.

    Args:
      base: 3-letter ISO code (e.g., "SGD")
      target: 3-letter ISO code (e.g., "JPY")

    Returns:
      JSON payload with the rate (from a free REST API), or {"error": "..."}.
    """
    urls = [
        f"https://hexarate.paikama.co/api/rates/latest/{base}?target={target}",
        f"https://api.exchangerate.host/latest?base={base}&symbols={target}",
    ]
    last_err = None
    for url in urls:
        try:
            r = requests.get(url, timeout=10)
            if r.status_code == 200:
                return r.json()
            last_err = f"HTTP {r.status_code}"
        except Exception as e:
            last_err = str(e)
    return {"error": f"FX fetch failed: {last_err or 'unknown'}"}
PY


#Tool: wiki_via_langchain — Wikipedia summary through LangChain

In [5]:
%%bash
set -e
cat > a2_tools_agent/wiki_via_langchain_adapter.py << 'PY'
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# Build the LC tool once and call it from a plain Python function
_wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1500))

def wiki_via_langchain(topic: str):
    """
    Use LangChain's WikipediaQueryRun to fetch a concise summary about a person/place/thing.
    ALWAYS prefer this tool for 'tell me about' or 'history of' queries.

    Args:
        topic: The topic/title to look up on Wikipedia (e.g., "Kyoto", "Mount Fuji").

    Returns:
        A concise plain-text summary (<= ~1500 chars), or {"error": "..."} on failure.
    """
    try:
        # LC tools typically accept a dict; some builds accept a string
        result = _wiki.invoke({"query": topic})
        return result if isinstance(result, str) else str(result)
    except Exception as e:
        return {"error": f"wiki_via_langchain failed: {e}"}
PY

echo "[OK] wrote wiki_via_langchain_adapter.py"


[OK] wrote wiki_via_langchain_adapter.py


#Tool: weather_outlook — 7-day forecast via Open-Meteo (no API key)


In [6]:
%%bash
set -e
cat > a2_tools_agent/custom_weather.py << 'PY'
import requests

_COUNTRY_ALIASES = {
    "japan": "JP",
    "united states": "US", "usa": "US", "u.s.": "US", "u.s.a.": "US",
    "united kingdom": "GB", "uk": "GB", "great britain": "GB", "england": "GB",
    "south korea": "KR", "korea": "KR",
    "germany": "DE", "france": "FR", "italy": "IT", "spain": "ES",
    "canada": "CA", "australia": "AU", "india": "IN", "singapore": "SG",
}

def _iso2(country: str | None) -> str | None:
    if not country:
        return None
    c = country.strip().lower()
    if len(c) == 2:
        return c.upper()
    return _COUNTRY_ALIASES.get(c)

def _choose_result(results, iso2: str | None, country_name: str | None):
    if not results:
        return None
    if iso2:
        for r in results:
            if str(r.get("country_code", "")).upper() == iso2:
                return r
    if country_name:
        cn = country_name.strip().lower()
        for r in results:
            if cn and cn in str(r.get("country", "")).strip().lower():
                return r
    return results[0]

def weather_outlook(city: str, country: str = "", days: int = 7):
    """
    Returns a concise 3–5 line outlook using Open-Meteo (no key).
    Args:
      city: e.g., "Tokyo"
      country: optional (e.g., "Japan" or "JP")
      days: up to 7
    Returns: {"source","summary","daily"} or {"error": "..."}
    """
    if not city or not city.strip():
        return {"error": "city is required"}

    iso2 = _iso2(country)
    q1 = city.strip()
    q2 = (city.strip() + " " + country.strip()).strip() if country else None

    def _geocode(q):
        r = requests.get("https://geocoding-api.open-meteo.com/v1/search",
                         params={"name": q, "count": 5}, timeout=10)
        if r.status_code != 200:
            return [], f"geocode HTTP {r.status_code}"
        return r.json().get("results") or [], None

    results, _ = _geocode(q1)
    if not results and q2:
        results, _ = _geocode(q2)
    if not results:
        return {"error": f"could not geocode '{q2 or q1}'"}

    chosen = _choose_result(results, iso2, country)
    if not chosen:
        return {"error": "geocode yielded no acceptable match"}

    lat, lon = chosen["latitude"], chosen["longitude"]
    city_name = chosen.get("name") or city

    d = max(1, min(int(days), 7))
    f = requests.get("https://api.open-meteo.com/v1/forecast",
                     params={
                        "latitude": lat, "longitude": lon,
                        "daily": "temperature_2m_max,temperature_2m_min,precipitation_probability_mean",
                        "timezone": "auto",
                     }, timeout=10)
    if f.status_code != 200:
        return {"error": f"forecast HTTP {f.status_code}"}
    fj = f.json()
    daily = []
    times = fj.get("daily", {}).get("time", []) or []
    tmaxs = fj.get("daily", {}).get("temperature_2m_max", []) or []
    tmins = fj.get("daily", {}).get("temperature_2m_min", []) or []
    pops  = fj.get("daily", {}).get("precipitation_probability_mean", []) or []
    for i in range(min(d, len(times))):
        daily.append({
            "date": times[i],
            "tmax": tmaxs[i] if i < len(tmaxs) else None,
            "tmin": tmins[i] if i < len(tmins) else None,
            "pop":  pops[i]  if i < len(pops)  else None,
        })
    if not daily:
        return {"error": "no daily data"}

    valid_tmax = [x["tmax"] for x in daily if x["tmax"] is not None]
    valid_tmin = [x["tmin"] for x in daily if x["tmin"] is not None]
    if not valid_tmax or not valid_tmin:
        return {"error": "missing temperature data"}

    tmax = max(valid_tmax)
    tmin = min(valid_tmin)
    wet = sum(1 for x in daily if (x.get("pop") or 0) >= 50)

    lines = [
        f"{city_name}: next {len(daily)} days ~ {round(tmin)}–{round(tmax)}°C.",
        f"{wet} day(s) with ≥50% precip probability." if wet else "Low chance of rain on most days.",
        "Source: Open-Meteo."
    ]
    return {"source": "Open-Meteo", "summary": " ".join(lines), "daily": daily}
PY

echo "[OK] custom_weather.py"


[OK] custom_weather.py


#Wire final agent — add tools & output policy (A2 travel assistant)


In [7]:
%%bash
set -e
cat > a2_tools_agent/agent.py << 'PY'
from google.adk.agents import Agent
from google.adk.tools import FunctionTool
from google.adk.tools.agent_tool import AgentTool

from .wiki_via_langchain_adapter import wiki_via_langchain
from .custom_weather import weather_outlook
from .custom_functions import get_fx_rate

# Optional: if you created a google_search_agent earlier, it will be used; otherwise ignored.
try:
    from .custom_agents import google_search_agent
except Exception:
    google_search_agent = None

tools = [
    FunctionTool(wiki_via_langchain),   # Wikipedia via LangChain (history/culture)
    FunctionTool(weather_outlook),      # Open-Meteo 7-day weather (no key)
    FunctionTool(get_fx_rate),          # FX JSON
]
if google_search_agent and len(getattr(google_search_agent, "tools", [])) > 0:
    tools.insert(1, AgentTool(agent=google_search_agent))  # optional delegate

root_agent = Agent(
    model='gemini-2.0-flash',
    name='a2_tools_agent',
    description='A2: Travel assistant with Wikipedia, Open-Meteo weather, and FX.',
    instruction=(
        "OUTPUT POLICY (must follow exactly):\n"
        "• If you used ANY tool, end with: 'Sources: <comma-separated names>' (e.g., Wikipedia, Open-Meteo).\n"
        "• If you did NOT use any tool, end with: 'Sources: none'.\n"
        "• For 'tell me about' / 'history of', you MUST call wiki_via_langchain first, then summarize.\n"
        "• For weather, call weather_outlook(city, country?) and summarize its 'summary' field.\n"
        "• For currency, call get_fx_rate and report either JSON or a numeric rate.\n"
        "Keep answers concise and factual."
    ),
    tools=tools,
)
PY

echo "[OK] final agent wired"


[OK] final agent wired


#Quick tests — Wikipedia, Weather, and FX tool calls


In [8]:
import subprocess, re, os

def run_agent(agent_pkg: str, prompt: str):
    p = subprocess.run(
        ["adk","run",agent_pkg],
        input=(prompt+"\n").encode(),
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
        check=False, env=dict(os.environ)
    )
    out = p.stdout.decode()
    m = re.search(rf"\[{re.escape(agent_pkg)}\]:\s*(.*)", out, flags=re.S)
    return (m.group(1).strip() if m else out), out

# Wikipedia test
print(run_agent("a2_tools_agent", "Tell me about the history of Kyoto (max 4 sentences).")[0])

# Weather test
print(run_agent("a2_tools_agent", "What is the weather outlook in Tokyo over the next week? keep it short.")[0])

# FX test
print(run_agent("a2_tools_agent", "What is the exchange rate from SGD to JPY? Return JSON.")[0])


Kyoto, officially Kyoto City, is the capital of Kyoto Prefecture in Japan. It was chosen in 794 as the new seat of Japan's imperial court by Emperor Kanmu and named Heian-kyō. The emperors of Japan ruled from Kyoto for eleven centuries until 1869, when the capital was moved to Tokyo after the Meiji Restoration. The modern municipality of Kyoto was established in 1889.
Sources: Wikipedia
[user]: 
Aborted!
The weather in Tokyo over the next 7 days will range from 10–22°C. There is one day with a 50% or greater probability of precipitation.
Sources: Open-Meteo

[user]: 
Aborted!
{"get_fx_rate_response": {"data": {"base": "SGD", "mid": 116.618, "target": "JPY", "timestamp": "2025-10-20T01:11:46.741Z", "unit": 1}, "status_code": 200}}
Sources: None
[user]: 
Aborted!
